In [ ]:
import pandas as pd
import dill

In [ ]:
with open('/Users/dolteanu/local_documents/Coding/MSc_github/Results/Nextstrain_clade_2/Nextstrain_clade_2_classification_report.pkl', 'rb') as data_path:
    classification_report = dill.load(data_path)

In [ ]:
classification_report.keys()

In [ ]:
# initialize an empty dictionary to store the average metrics for each classifier
average_metrics = {}

# iterate over the classifiers
for classifier_name, reports in classification_report.items():
    # initialize an empty dictionary to store the average metrics for each class
    average_class_metrics = {}
    # iterate over the classification reports (list) for each fold
    for report in reports:
        for class_name, metrics in report.items():
            # if class_name!='accuracy':
                # metrics.pop('support')
                if class_name not in average_class_metrics:
                    average_class_metrics[class_name] = metrics
                elif class_name!='accuracy':
                    for metric, value in metrics.items():
                        average_class_metrics[class_name][metric] += value
                else:
                    average_class_metrics['accuracy'] += metrics
    # divide the accumulated values by the number of folds to obtain the average metrics
    num_folds = len(reports)
    for class_name, metrics in average_class_metrics.items():
        if class_name!='accuracy':
            for metric, value in metrics.items():
                average_class_metrics[class_name][metric] /= num_folds
        else:
            average_class_metrics['accuracy'] /= num_folds
    average_metrics[classifier_name] = average_class_metrics

In [ ]:
df = pd.concat({classifier: pd.DataFrame(report).T for classifier, report in average_metrics.items()}, axis=0)
df.drop(columns='support')
df.drop(labels='accuracy', level=1)

In [ ]:
with open('nextstrain_clade.tex','w') as tf:
     tf.write(df.to_latex(float_format="%.2f"))

In [ ]:
accuracy_dict = {model: average_metrics[model]['accuracy'] for model in average_metrics}
accuracy_df = pd.DataFrame(list(accuracy_dict.items()), columns=['Model', 'Unbalanced Accuracy'])
accuracy_table = accuracy_df.to_latex(index=False)
with open('nextstrain_clade_accuracy.tex','w') as tf:
     tf.write(accuracy_df.to_latex(index=False,float_format="%.2f"))
